In [ ]:
import os
os.chdir('..')

import warnings
warnings.filterwarnings("ignore")

# Feature Engineering

* The goal of this notebook is to show examples of how to create item and user features. 
* Mab2Rec is _independent_ of item, user, and interaction data used in recommendations and assumes that input data is created before building recommenders. 
* Sample input is given in `data/` which includes user features in `features_user.csv` and item features in `features_item.csv`.
* This notebook shows examples of how to create user or item features from **structured**, **unstructured**, and **sequential** data.
* In addition to techniques covered here, and you can utilize any other source to create your input data. 
* An overview of these libraries is [presented at All Things Open 2021](https://www.youtube.com/watch?v=54d_YUalvOA).

# Table of Contents

1. [Structured Data via Selective](#Structured-Data-via-Selective)
2. [Unstructured Data via TextWiser](#Unstructured-Data-via-TextWiser)
3. [Sequential Data via Seq2Pat](#Sequential-Data-via-Seq2Pat)

# Structured Data via Selective

* The most common data source is structured, tabular data. 
* In recommenders, the typical usage of structured data is to represent **user features**.
* When there are many user features to consider, feature selection can decide which features to include in the user context.
* For feature selection, you can leverage [Selective](https://github.com/fidelity/selective).
* Selective provides an easy-to-use API for supervised and unsupervised feature selection methods.
* In unsupervised fashion, given a set of users, important features can be identified according to variance, correlation and statistical measures. 
* In supervised fashion, given a set of users _and_ the interaction label (e.g., click on _any item_), important features can be identified according to a linear or non-linear model. 
* Let's install the library and explore a quick start example.

In [ ]:
# Install Selective
!pip install -q selective;

In [ ]:
# Import Selective and SelectionMethod
from sklearn.datasets import load_boston
from feature.utils import get_data_label
from feature.selector import Selective, SelectionMethod

# Data
data, label = get_data_label(load_boston())

# Feature selectors from simple to more complex
selector = Selective(SelectionMethod.Variance(threshold=0.0))
selector = Selective(SelectionMethod.Correlation(threshold=0.5, method="pearson"))
selector = Selective(SelectionMethod.Statistical(num_features=3, method="anova"))
selector = Selective(SelectionMethod.Linear(num_features=3, regularization="none"))
selector = Selective(SelectionMethod.TreeBased(num_features=3))

# Feature reduction
subset = selector.fit_transform(data, label)
print("Reduction:", list(subset.columns))
print("Scores:", list(selector.get_absolute_scores()))

* In this example, we show 5 different `selector` methods. 
* Any selection approach can be used to `fit_transform` the dataset. 
* A more robust approach is to apply different selectors, and then to select feautures that are deemed important by several selectors. 
* It is even better to repeat this within cross-validation to make sure the selection is stable. 
* Selective offers a benchmarking utility to achieve this. 
* See [Selective Benchmarking](https://github.com/fidelity/selective#benchmarking).

# Unstructured Data via TextWiser

* Unstructured data is another common data source utilizing text, audio, and video features.   
* In recommenders, the typical usage of unstructured data is to represent **item features**.
* Unstructured data should first be featurized before consumption in recommenders.  
* For text data, you can leverage [TextWiser](https://github.com/fidelity/textwiser) to create text embeddings of item representations.
* TextWiser ([AAAI'21](https://ojs.aaai.org/index.php/AAAI/article/view/17814)) provides an easy-to-use API for a rich set of text featurization methods and their transformation while taking advantage of state-of-the-art pretrained NLP models.
* Let's install the library and explore a quick start example.

In [ ]:
# Install TextWiser
!pip install -q textwiser;

In [ ]:
# Conceptually, TextWiser is composed of an Embedding, potentially with a pretrained model,
# that can be chained into zero or more Transformations
from textwiser import TextWiser, Embedding, Transformation, WordOptions, PoolOptions
from transformers import logging
logging.set_verbosity_error()

# Data
documents = ["Some document", "More documents. Including multi-sentence documents."]

# Model: TFIDF `min_df` parameter gets passed to sklearn automatically
emb = TextWiser(Embedding.TfIdf(min_df=1))

# Model: TFIDF followed with an NMF + SVD
emb = TextWiser(Embedding.TfIdf(min_df=1), [Transformation.NMF(n_components=30), Transformation.SVD(n_components=10)])

# Model: Word2Vec with no pretraining that learns from the input data
emb = TextWiser(Embedding.Word(word_option=WordOptions.word2vec, pretrained=None), Transformation.Pool(pool_option=PoolOptions.min))

# Model: BERT with the pretrained bert embedding
emb = TextWiser(Embedding.Word(word_option=WordOptions.bert), Transformation.Pool(pool_option=PoolOptions.first))

# Features
vecs = emb.fit_transform(documents)

print(documents)
print(vecs)

* In this example, we show different embeddings from simple `TFIDF` to more complex `BERT`. 
* Notice how the `Embedding` can be followed with one more transformation operations, such as `NMF` or `SVD`. 
* In general, the `Transformation` reduces the dimensionality of the text representation to create succint embeddings. 
* Running `fit_transform` on the documents return the embedding of each document. 
* Checkout different word options, pre-trained models, and other transformations.
* See the rich list of [TextWiser Embeddings](https://github.com/fidelity/textwiser#available-embeddings).

# Sequential Data via Seq2Pat

* For most applications, structured data for user features via Selective and unstructured data for item features via TextWiser will suffice.
* In applications where events occur in sequences, time horizon can serve as another source of information to build advanced recommenders.
* We suggest starting simple and consider such sequential data to improve upon initial results once an end-to-end benchmarking is completed.
* In recommenders, the typical usage of sequential data is to capture the behaviour of a user over time as part of **user features**. 
* For sequential data, you can leverage [Seq2Pat](https://github.com/fidelity/seq2pat).
* Seq2Pat ([AAAI'22](https://aaai.org/Conferences/AAAI-22/)) provides an easy-to-use API for frequent pattern mining in sequential datasets. 
* We first find frequent patterns with desired properties.
* Then, we represent each user with a one-hot vector denoting the existence of frequent patterns in user's sequential behaviour. This one-hot features can be used to augment user representation.
* Let's install the library and explore a quick start example.

In [ ]:
# Install Seq2Pat
!pip install -q seq2pat;

In [ ]:
# Example to show how to find frequent sequential patterns
# from a given sequence database subject to constraints
# and generate one-hot encodings as the features of each user
from sequential.seq2pat import Seq2Pat, Attribute
from sequential.dpm import get_one_hot_encodings

# Sequences database with 3 sequences
sequences = [["A", "A", "B", "A", "D"],
             ["C", "B", "A"],
             ["C", "A", "C", "D"]]

# Prices of each item in sequences
values = [[5, 5, 3, 8, 2],
          [1, 3, 3],
          [4, 5, 2, 1]]

# Seq2Pat over 3 sequences
seq2pat = Seq2Pat(sequences=sequences)

# Price attribute corresponding to each item
price = Attribute(values=values)

# Average price constraint
seq2pat.add_constraint(3 <= price.average() <= 4)

# Patterns that occur at least twice (A-D)
patterns = seq2pat.get_patterns(min_frequency=2)
print("Frequent Patterns: ", patterns)

# Encoding of each user with respect to frequent patterns (A-D)
encodings = get_one_hot_encodings(sequences, patterns)
print("Encodings:\n", encodings)

* In this example, we have 3 users with certain sequential events, e.g. page visits in-order. 
* Notice, how the length of each sequence in the sequence database can be different.
* We can mine for frequent patterns in this sequence database. The `min_frequency` threshold denotes the minimum number of occurrence of a pattern to be considered frequent.
* More importantly, we consider **attributes** that correspond to each sequential event. Here, the price of the item in each page visit.
* Then, we add **constraints** to reason about attributes. Here, the average price to be between 3 and 4.
* Pattern mining operates on the sequence database and seeks frequent patterns that meet the minimum frequency threshold and satisfy constraints.
* See other [Seq2Pat Constraints](https://github.com/fidelity/seq2pat/blob/master/notebooks/usage_example.ipynb) for .
* Finally, we create one-hot encodings of each user with respect to frequent pattern (A-D). The one-hot vector indicates the existence of such pattern in the user's sequence.
* These one-hot encodings of users can be used as input features for the recommendation model.
* In this example, we mined patterns for the entire population. A more advanced approach is to mine patterns that distinguish between the positive vs. negative behavior, e.g., differentiate users who engaged with a certain content from those who do not.
* To that end, see [Dichotomic Pattern Mining (DPM)](https://github.com/fidelity/seq2pat/blob/master/notebooks/dichotomic_pattern_mining.ipynb) from [AAAI'22](https://arxiv.org/abs/2201.09178) that exploits the dichotomy of such behavior to patterns unique to each cohort.